# Import necessary depencencies

In [ ]:
import numpy as np
from keras.layers import Dropout, Activation, Dense
from keras.models import Sequential
import model_evaluation_utils as meu
import utils
import matplotlib.pyplot as plt
from sklearn import metrics
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, Dropout, SpatialDropout1D
from keras.layers import LSTM
import xgboost as xgb
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import itertools
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from mlxtend.plotting import plot_learning_curves
from mlxtend.plotting import plot_decision_regions
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
import pandas as pd
from keras.regularizers import l2
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.advanced_activations import PReLU
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

np.set_printoptions(precision=2, linewidth=80)
NWORKERS=15

# Load normalized data

In [ ]:
# train and test datasets
train_reviews = utils.readFromDisk('train_reviews')
train_sentiments = utils.readFromDisk('train_sentiments')
test_reviews = utils.readFromDisk('test_reviews')
test_sentiments = utils.readFromDisk('test_sentiments')
tokenized_train = utils.readFromDisk('tokenized_train')
tokenized_test = utils.readFromDisk('tokenized_test')
train_sentiments_encoded = utils.readFromDisk('train_sentiments_encoded')
test_sentiments_encoded = utils.readFromDisk('test_sentiments_encoded')

#glove features
# feature engineering with GloVe model
train_glove_features = utils.readFromDisk('train_glove_features')
test_glove_features = utils.readFromDisk('test_glove_features')


In [ ]:
#split train into train and vaildate sets for models that need it
def splitTrainForValidation(train_x,train_y,val_ratio):
    X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, test_size=val_ratio)
    return X_train, X_val, y_train, y_val


# Logistic Regression (hyperparameter tuned and cached)
# #######################################################################################################33333

In [ ]:
try:
    lr=utils.readFromDisk('lr')
except FileNotFoundError as te:
    #lr0 = LogisticRegression(max_iter=1000,n_jobs=NWORKERS)
    lr0 = LogisticRegression()

    lr_n_jobs=[NWORKERS]
    lr_max_iter=[1000]
    lr_penalty = ['l2']
    lr_C = np.logspace(0, 4, 10)
    # Create hyperparameter options
    lr_hyperparameters = dict(max_iter=lr_max_iter,C=lr_C, penalty=lr_penalty,n_jobs=lr_n_jobs)

    # Create grid search using 5-fold cross validation
    #had to set njobs here to nworkers here because njobs on the algo above was not triggering multiproc
    lr_clf = GridSearchCV(lr0, lr_hyperparameters,cv=5, n_jobs=NWORKERS,scoring = 'roc_auc')

    # Fit grid search
    lr = lr_clf.fit(train_glove_features,train_sentiments)

    utils.writeToDisk(lr,'lr')

In [ ]:
    # View best hyperparameters
    lr.best_estimator_

In [ ]:
# Predict target vector
lr_predictions=lr.predict(test_glove_features)



## Logistic Regression - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=lr_predictions,
                                      classes=[1,0])

# SVM using sgd classifier (hyperparameter tuned and cached)
# #######################################################################################################33333

In [ ]:
try:
    svm=utils.readFromDisk('svm')
except FileNotFoundError as te:
    #svm0 = SGDClassifier(max_iter=500,n_jobs=NWORKERS)
    svm0 = SGDClassifier()

    svm_n_jobs=[NWORKERS]
    svm_max_iter=[1000]
    svm_loss= ["hinge"]
    svm_alpha = [0.0001, 0.001, 0.01, 0.1]
    svm_penalty = ["l2"]
    svm_hyperparameters = dict(max_iter=lr_max_iter,loss=svm_loss, alpha=svm_alpha,penalty=svm_penalty,n_jobs=svm_n_jobs)

    svm_clf = GridSearchCV(svm0, svm_hyperparameters,cv=5, n_jobs=1,scoring = 'roc_auc')

    svm = svm_clf.fit(train_glove_features,train_sentiments)

    utils.writeToDisk(svm,'svm')


In [ ]:
    svm.best_estimator_

In [ ]:
svm_predictions=svm.predict(test_glove_features)

## SVM - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=svm_predictions,
                                      classes=[1,0])


# XGBoost (hyperparameter tuned and cached)
# #######################################################################################################33333

In [ ]:
try:
    xg=utils.readFromDisk('xg')
except FileNotFoundError as te:
#    xg = xgb.XGBClassifier(n_estimators=500, max_depth=5, base_score=0.5,objective='binary:logistic', 
#                       random_state=42,nthread=16,n_jobs=NWORKERS)
    xg0 = xgb.XGBClassifier()

    xg_max_depth=[10]
    xg_n_estimators=[200]
    xg_learning_rate=[0.1, 0.01, 0.05]
    xg_base_score=[0.5]
    xg_objective=['binary:logistic']
    xg_random_state=[42]
    xg_n_thread=[1]
    xg_n_jobs=[NWORKERS]
    xg_hyperparameters = dict(max_depth=xg_max_depth,n_estimators=xg_n_estimators, 
                              learning_rate=xg_learning_rate, base_score=xg_base_score,
                              objective=xg_objective,random_state=xg_random_state,
                              n_thread=xg_n_thread,n_jobs=xg_n_jobs)

    xg_clf = GridSearchCV(xg0, xg_hyperparameters,cv=5, n_jobs=1,scoring = 'roc_auc')
    
    xg=xg_clf.fit(train_glove_features, train_sentiments)

    utils.writeToDisk(xg,'xg')


In [ ]:
    xg.best_estimator_

In [ ]:
xg_predictions= xg.predict(test_glove_features)


## XGBoost - Performance


In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=xg_predictions, 
                                      classes=[1,0])  


# Deep neural network architecture (DNN) (hyperparameter tuned and cached)
# #######################################################################################################33333

In [ ]:
def construct_deepnn_architecture(num_input_features):
    dnn_model = Sequential()
    dnn_model.add(Dense(512, activation='relu', input_shape=(num_input_features,)))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(512, activation='relu'))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(512, activation='relu'))
    dnn_model.add(Dropout(0.2))
    dnn_model.add(Dense(2))
    dnn_model.add(Activation('softmax'))

    dnn_model.compile(loss='categorical_crossentropy', optimizer='adam',                 
                      metrics=['accuracy'])
    return dnn_model
def build_keras_base(hidden_layers = [64, 64, 64], dropout_rate = 0, 
                     l2_penalty = 0.1, optimizer = 'adam',
                     n_input = 100, n_class = 2):
    """
    Keras Multi-layer neural network. Fixed parameters include: 
    1. activation function (PRelu)
    2. always uses batch normalization after the activation
    3. use adam as the optimizer
    
    Parameters
    ----------
    Tunable parameters are (commonly tuned)
    
    hidden_layers: list
        the number of hidden layers, and the size of each hidden layer
    
    dropout_rate: float 0 ~ 1
        if bigger than 0, there will be a dropout layer
    
    l2_penalty: float
        or so called l2 regularization
    
    optimizer: string or keras optimizer
        method to train the network
    
    Returns
    -------
    model : 
        a keras model

    Reference
    ---------
    https://keras.io/scikit-learn-api/
    """   
    model = Sequential()   
    for index, layers in enumerate(hidden_layers):       
        if not index:
            # specify the input_dim to be the number of features for the first layer
            model.add(Dense(layers, input_dim = n_input, kernel_regularizer = l2(l2_penalty)))
        else:
            model.add(Dense(layers, kernel_regularizer = l2(l2_penalty)))
        
        # insert BatchNorm layer immediately after fully connected layers
        # and before activation layer
        model.add(BatchNormalization())
        model.add(PReLU())        
        if dropout_rate:
            model.add(Dropout(p = dropout_rate))
    
    model.add(Dense(n_class))
    model.add(Activation('softmax'))
    
    # the loss for binary and muti-class classification is different 
    loss = 'binary_crossentropy'
    if n_class > 2:
        loss = 'categorical_crossentropy'
    
    model.compile(loss = loss, optimizer = optimizer, metrics = ['accuracy'])   
    return model


## DNN - Training and Prediction

In [ ]:

try:   
    dnn=utils.readFromDisk('dnn')
except FileNotFoundError as te:
    #from http://ethen8181.github.io/machine-learning/keras/nn_keras_hyperparameter_tuning.html
    dnn0 = KerasClassifier(build_fn = construct_deepnn_architecture(num_input_features=300)    )
    # random search's parameter:
    # specify the options and store them inside the dictionary
    # batch size and training method can also be hyperparameters, 
    # but it is fixed
    dnn_dropout_rate_opts  = [0, 0.2, 0.5]
    dnn_hidden_layers_opts = [[64, 64, 64, 64], [32, 32, 32, 32, 32], [100, 100, 100]]
    dnn_l2_penalty_opts = [0.01, 0.1, 0.5]
    dnn_hyperparameters = dict(hidden_layers=dnn_hidden_layers_opts,dropout_rate=dnn_dropout_rate_opts,
                               l2_penalty=dnn_l2_penalty_opts)

    dnn_epochs=[5]
    dnn_batch_size=[100]
    dnn_validation_split=[0.1]
    dnn_shuffle=[True]
    dnn_workers=[NWORKERS]
    dnn_use_multiprocessing=[True]
    dnn_fit_parameters = dict(epochs=dnn_epochs, batch_size=dnn_batch_size,validation_split=dnn_validation_split,
                               shuffle=dnn_shuffle, workers=dnn_workers, use_multiprocessing=dnn_use_multiprocessing)
    
    dnn_clf = RandomizedSearchCV(
        dnn0, 
        param_distributions = dnn_hyperparameters,
        scoring = 'neg_log_loss',
        n_iter = 3, 
        cv = 5,
        n_jobs = NWORKERS,
        verbose = 1
    )    
    
    '''   
    
    dnn0 = construct_deepnn_architecture(num_input_features=300)
    k_dnn0 = KerasClassifier(dnn0, verbose=0)

    dnn_epochs=[5]
    dnn_batch_size=[100]
    dnn_validation_split=[0.1]
    dnn_shuffle=[True]
    dnn_workers=[NWORKERS]
    dnn_use_multiprocessing=[True]
    #dnn_dropout_rate_opts  = [0, 0.2, 0.5]
    #dnn_hidden_layers_opts = [[64, 64, 64, 64], [32, 32, 32, 32, 32], [100, 100, 100]]
    #dnn_l2_penalty_opts = [0.01, 0.1, 0.5]
    dnn_hyperparameters = dict(epochs=dnn_epochs, batch_size=dnn_batch_size,validation_split=dnn_validation_split,
                               shuffle=dnn_shuffle, workers=dnn_workers, use_multiprocessing=dnn_use_multiprocessing)

    #batch_size = 100
    #dnn.fit(train_glove_features, train_sentiments_encoded, epochs=5, batch_size=batch_size, 
    #        shuffle=True, validation_split=0.1, verbose=1,workers=NWORKERS,use_multiprocessing=True)
    dnn_clf = RandomizedSearchCV(
        k_dnn0, 
        dnn_hyperparameters,
        scoring = 'neg_log_loss',
        cv = 5,
        n_jobs = 1)
    '''
    
    dnn=dnn_clf.fit(train_glove_features,train_sentiments_encoded,fit_params=dnn_fit_parameters)    
    
    utils.writeToDisk(dnn,'dnn')


In [ ]:
le = utils.readFromDisk('label_encoder')
y_pred = dnn.predict_classes(test_glove_features)
dnn_predictions = le.inverse_transform(y_pred) 

## DNN - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=dnn_predictions, 
                                      classes=[1,0])  


# LSTM
# #######################################################################################################33333


In [ ]:
from collections import Counter

# build word to index vocabulary
token_counter = Counter([token for review in tokenized_train for token in review])
vocab_map = {item[0]: index+1 for index, item in enumerate(dict(token_counter).items())}
max_index = np.max(list(vocab_map.values()))
vocab_map['PAD_INDEX'] = 0
vocab_map['NOT_FOUND_INDEX'] = max_index+1
vocab_size = len(vocab_map)
# view vocabulary size and part of the vocabulary map
print('Vocabulary Size:', vocab_size)
print('Sample slice of vocabulary map:', dict(list(vocab_map.items())[10:20]))

## LSTM- Encode and Pad datasets & Encode prediction class labels

In [ ]:
# get max length of train corpus and initialize label encoder
max_len = np.max([len(review) for review in tokenized_train])

## Train reviews data corpus
# Convert tokenized text reviews to numeric vectors
train_X = [[vocab_map[token] for token in tokenized_review] for tokenized_review in tokenized_train]
train_X = sequence.pad_sequences(train_X, maxlen=max_len) # pad 

## Test reviews data corpus
# Convert tokenized text reviews to numeric vectors
test_X = [[vocab_map[token] if vocab_map.get(token) else vocab_map['NOT_FOUND_INDEX'] 
           for token in tokenized_review] 
              for tokenized_review in tokenized_test]
test_X = sequence.pad_sequences(test_X, maxlen=max_len)

# view vector shapes
print('Max length of train review vectors:', max_len)
print('Train review vectors shape:', train_X.shape, ' Test review vectors shape:', test_X.shape)

## LSTM- Build, train and visualize the LSTM Model

In [ ]:
EMBEDDING_DIM = 128 # dimension for dense embeddings for each token
LSTM_DIM = 64 # total LSTM units

lstm = Sequential()

try:   
    lstm=utils.readFromDisk('lstm')
    print(lstm.summary())
except FileNotFoundError as te:    
    lstm.add(Embedding(input_dim=vocab_size, output_dim=EMBEDDING_DIM, input_length=max_len))
    lstm.add(SpatialDropout1D(0.2))
    lstm.add(LSTM(LSTM_DIM, dropout=0.2, recurrent_dropout=0.2))
    lstm.add(Dense(2, activation="sigmoid"))
    
    lstm.compile(loss="binary_crossentropy", optimizer="adam",
                  metrics=["accuracy"])
    print(lstm.summary())

    SVG(model_to_dot(lstm, show_shapes=True, show_layer_names=False, 
                     rankdir='LR').create(prog='dot', format='svg'))
    batch_size = 100
    lstm.fit(train_X, train_sentiments_encoded, epochs=2, batch_size=batch_size, 
              shuffle=True, validation_split=0.1, verbose=1,workers=NWORKERS,use_multiprocessing=True)

    utils.writeToDisk(lstm,'lstm')


## LSTM - Predictions

In [ ]:
lstm_pred_test = lstm.predict_classes(test_X)

In [ ]:
lstm_predictions = le.inverse_transform(lstm_pred_test)

## LSTM - Performance

In [ ]:
meu.display_model_performance_metrics(true_labels=test_sentiments, predicted_labels=lstm_predictions, 
                                      classes=[1,0])  

# Ensemble Stacking
# #######################################################################################################33333

In [ ]:
clf1 = KNeighborsClassifier(n_neighbors=1,n_jobs=NWORKERS)
clf2 = RandomForestClassifier(random_state=1,n_jobs=NWORKERS)
clf3 = GaussianNB()
lr = LogisticRegression(penalty='l2', max_iter=500, C=1,n_jobs=NWORKERS)
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3], 
                          meta_classifier=lr)

## Ensemble Stacking - Predictions via n fold validations

In [ ]:
clf_list = [clf1, clf2, clf3, sclf]
#preds_list=[]

try:
    clf_list=utils.readFromDisk('clf_list')
    #preds_list=utils.readFromDisk('preds_list')
except FileNotFoundError as te:   
    clf_list = [clf1, clf2, clf3, sclf]
    preds_list=[]
    for clf in clf_list:
        clf.fit(train_glove_features,train_sentiments)
        #pred = cross_val_predict(clf, train_glove_features, train_sentiments, cv=2,n_jobs=NWORKERS)
        #preds_list.append(pred)
    
    utils.writeToDisk(clf_list,'clf_list')
    #utils.writeToDisk(preds_list,'preds_list')

In [ ]:
try:
    knn_predictions=utils.readFromDisk('knn_predictions')
except FileNotFoundError as te:    
    knn_predictions=clf1.predict(test_glove_features)
    utils.writeToDisk(knn_predictions,'knn_predictions')
try:
    rf_predictions=utils.readFromDisk('rf_predictions')
except FileNotFoundError as te:    
    rf_predictions=clf2.predict(test_glove_features)
    utils.writeToDisk(rf_predictions,'rf_predictions')

try:
    gau_predictions=utils.readFromDisk('gau_predictions')
except FileNotFoundError as te:    
    gau_predictions=clf3.predict(test_glove_features)
    utils.writeToDisk(gau_predictions,'gau_predictions')

try:
    stack_predictions=utils.readFromDisk('stack_predictions')
except FileNotFoundError as te:    
    stack_predictions=sclf.predict(test_glove_features)
    utils.writeToDisk(stack_predictions,'stack_predictions')
    


# All Models Evaluation
# #######################################################################################################33333

## Model ROC curves

In [ ]:
plt.figure(0).clf()

color = ['blue', 'orange', 'red', 'green', 'coral',
             'grey', 'indigo', 'gold', 'lime', 'olive',
             'pink', 'navy', 'magenta', 'yellow', 'tomato',
             'turquoise', 'yellowgreen', 'maroon', 'lightblue']
mlr=[]
msvm=[]
mdnn=[]
mxg=[]
mlstm=[]
mknn=[]
mrf=[]
mgau=[]
mstack=[]

def metricsAndROC(pred,metricsArray,rocTitle,colorIndex):
    fpr, tpr, thresholds = metrics.roc_curve(test_sentiments, pred)
    auc = metrics.roc_auc_score(test_sentiments, pred)
    metricsArray.append(metrics.f1_score(test_sentiments, pred))
    metricsArray.append(metrics.precision_score(test_sentiments, pred))
    metricsArray.append(metrics.accuracy_score(test_sentiments, pred))
    metricsArray.append(metrics.recall_score(test_sentiments, pred))
    
    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr,color=color[colorIndex], label=rocTitle.format(auc))



metricsAndROC(lr_predictions,mlr,'LR',0)
metricsAndROC(svm_predictions,msvm,'SVM',1)
metricsAndROC(dnn_predictions,mdnn,'DNN',2)
metricsAndROC(xg_predictions,mxg,'XGBoost',3)
metricsAndROC(lstm_predictions,mlstm,'LSTM',4)
metricsAndROC(knn_predictions,mknn,'KNN',5)
metricsAndROC(rf_predictions,mrf,'RFOR',6)
metricsAndROC(gau_predictions,mgau,'GAUS',7)
metricsAndROC(stack_predictions,mstack,'ENSEM-STACK',8)



#show the roc curve now
# axis labels
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
# show the legend
plt.legend(loc='center right')
plt.show()

## All Models Metrics comparison

In [ ]:
n_groups = 4
index = np.arange(n_groups)
bar_width = .1

plt.bar(index,mlr, bar_width, color=color[0], label='LR')


z=index + bar_width
plt.bar(z, msvm, bar_width, color=color[1],label='SVM')


z=z+ bar_width
plt.bar(z, mdnn, bar_width, color=color[2], label='DNN')

z=z+ bar_width
plt.bar(z,mxg , bar_width, color=color[3], label='XGB')

z=z+ bar_width
plt.bar(z,mlstm , bar_width,color=color[4], label='LSTM')



#ax.set_xlabel('Metric')
#ax.set_ylabel('Value')
#ax.set_title('Comparison of Feature Engineering Models on Amazon Reviews')
#ax.set_xticks(index + bar_width / 2)
pltLabels=['F1','PRECISION','ACCURACY','RECALL']
# Add xticks on the middle of the group bars
plt.xlabel('Model Metrics', fontweight='bold')
plt.xticks([r + bar_width for r in range(n_groups)], pltLabels)
 
# Create legend & Show graphic
plt.legend(frameon=False,ncol=3, loc='lower left')
plt.show()
